In [1]:
import pickle
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from sklearn.utils import shuffle
from datetime import datetime 

from keras.models import Model 
from keras.layers import Input, Embedding, Dot, Add, Flatten 
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

2023-04-05 16:39:20.739001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
df = pd.read_csv("./data/edited_ratings.csv")
df['userId'] = df['userId'] + 1
df

,Unnamed: 0,userId,movieId,rating,movie_idx
0,0,0,2,3.5,2
1,1,0,29,3.5,29
2,2,0,32,3.5,32
3,3,0,47,3.5,47
4,4,0,50,3.5,50
...,...,...,...,...,...
20000258,20000258,138492,68954,4.5,13821
20000259,20000259,138492,69526,4.5,13929
20000260,20000260,138492,69644,3.0,13942
20000261,20000261,138492,70286,5.0,14060


In [14]:
N = df.userId.max() + 1 # number of users 
M = df.movie_idx.max() + 1 # number of movies 

In [15]:
# split in train and test 
df = shuffle(df) 
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff] 
df_test = df.iloc[cutoff:] 

In [16]:
# init vars 
K = 10 # latent dimensionality 
mu = df_train.rating.mean() 
epoch = 25
reg = 0 # regularization penalty

In [17]:
# keras model 
u = Input(shape=(1, ))
m = Input(shape=(1, ))
u_embedding = Embedding(N, K, embeddings_regularizer=l2(reg))(u) # (N, 1, K)  
m_embedding = Embedding(M, K, embeddings_regularizer=l2(reg))(m) # (M, 1, K)

In [18]:
u_bias = Embedding(N, 1, embeddings_regularizer=l2(reg))(u) # (N, 1, 1)
m_bias = Embedding(M, 1, embeddings_regularizer=l2(reg))(m) # (N, 1, 1) 
x = Dot(axes=2)([u_embedding, m_embedding])

In [19]:
x = Add()([x, u_bias, m_bias])
x = Flatten()(x) # (N, 1)

In [20]:
model = Model(inputs=[u, m], outputs=x) 
model.compile(loss='mse', optimizer=SGD(lr=0.01, momentum=0.9), metrics=['mse'])

/Users/faa/anaconda3/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [21]:
r = model.fit(
    x = [df_train.userId.values, df_train.movie_idx.values], 
    y = [df_train.rating.values - mu], 
    epochs=epoch, 
    batch_size=128, 
    validation_data=(
    [df_test.userId.values, df_test.movie_idx.values], 
    df_test.rating.values - mu
    )
)

Epoch 1/25
125002/125002 [==============================] - 293s 2ms/step - loss: 0.8691 - mse: 0.8691 - val_loss: 0.8150 - val_mse: 0.8150
Epoch 2/25
125002/125002 [==============================] - 292s 2ms/step - loss: 0.7966 - mse: 0.7966 - val_loss: 0.7847 - val_mse: 0.7847
Epoch 3/25
125002/125002 [==============================] - 313s 3ms/step - loss: 0.7743 - mse: 0.7743 - val_loss: 0.7702 - val_mse: 0.7702
Epoch 4/25
 66476/125002 [==============>...............] - ETA: 2:10 - loss: 0.7640 - mse: 0.7640

KeyboardInterrupt: 

In [ ]:
# plot losses 
plt.plot(r.history['loss'], label = "train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()

In [ ]:
# plot mse 
plt.plot(r.history['mean_squared_error'], label="train mse")
plt.plot(r.history['val_mean_sqared_error'], label='test mse')
plt.legend() 
plt.show()